# Using Functional API to build CNN

We start introducing the concept of Functional API as an alternative way of building keras models. In the previous examples on MLP and CNN on MNIST, we used Sequential API. The Sequential API is fine if we are building simple models wherein there is a single point for input and single point for output. In advanced models, this is not sufficient since we build more complex graphs possibly with multiple inputs and outputs. In such cases, Functional API is the method of choice.

Functional API builds upon the concept of function composition:

\begin{equation*}
y = f_n \circ f_{n-1} \circ \ldots \circ f_1(x)
\end{equation*}

The output of one function becomes the input of the next function and so on. We can also have a function with multiple outputs that become inputs to multiple functions. Or, we can have multiple functional blocks with multiple separate inputs that are combined into one or more outputs. 

In the following example, we will show how to build a model made of `3-Conv2D-1-Dense` using Functional API. 

Similar to the previous example on CNN on MNIST, let us do the initializations, and input and label pre-processing.

In [25]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

### Hyper-parameters

The hyper-parameters are similar to the one used in CNN on MNIST example.

In [26]:
# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64

## Actual Model Building

Using the Functional API Keras layer `y = Conv2D(x)`, we can stack 3 CNN layers together to form a simple backbone network. The `y = MaxPooling2D(x)` is used to compress the learned feature maps. With compressed feature maps, the CNN learns new representations with a bigger receptive field.

In Functional API, the output of one layer becomes the input of the next layer. For example if the first layer is `y2 = Conv2D(y1)`, then the next layer is `y3 = Conv2D(y2)`. To save from variable name pollution, we normally reuse the same variable name (eg. `y`) as shown below.

In Sequential model building, we use the `add()` method of a model to stack multiple layers together.

In [27]:
# use functional API to build cnn layers
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu',
           padding='same')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu',
           padding='same')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu',
           padding='same')(y)

### Head is a Dense Layer

Since we are doing logistic regression, we need to `flatten` the output of the 3-layer CNN so that we can generate the right number of logits to model a 10-class categorical distribution. This is the same concept that we used in MLP.

In [28]:
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
#y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)
# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)
# network model in text
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 3136)             

### Model Training and Validation

The last step is similar to our MLP example. We compile the model and perform training by calling `fit`. 

In [29]:
# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# train the model with input images and labels
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=20,
          batch_size=batch_size)

# model accuracy on test dataset
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Epoch 1/20
469/469 [==============================] - 46s 98ms/step - loss: 1.4132 - accuracy: 0.5942 - val_loss: 0.4621 - val_accuracy: 0.8452
Epoch 2/20
469/469 [==============================] - 44s 93ms/step - loss: 0.3605 - accuracy: 0.8910 - val_loss: 0.2893 - val_accuracy: 0.9168
Epoch 3/20
469/469 [==============================] - 44s 94ms/step - loss: 0.2649 - accuracy: 0.9204 - val_loss: 0.2212 - val_accuracy: 0.9346
Epoch 4/20
469/469 [==============================] - 44s 94ms/step - loss: 0.2087 - accuracy: 0.9378 - val_loss: 0.1718 - val_accuracy: 0.9486
Epoch 5/20
469/469 [==============================] - 44s 95ms/step - loss: 0.1693 - accuracy: 0.9498 - val_loss: 0.1573 - val_accuracy: 0.9511
Epoch 6/20
469/469 [==============================] - 44s 93ms/step - loss: 0.1407 - accuracy: 0.9583 - val_loss: 0.1250 - val_accuracy: 0.9625
Epoch 7/20
469/469 [==============================] - 44s 94ms/step - loss: 0.1200 - accuracy: 0.9647 - val_loss: 0.1015 - val_accuracy: